In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dedek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import gensim
import os
import copy
import datetime
import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from wordcloud import WordCloud
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import contextlib
import spacy
import re
import os
import random
import math
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
tweet_tokenizer = TweetTokenizer()

In [9]:
slang = pd.read_csv("../Data/slang-corpus.txt", sep='\s*,\s*') 
slang.head(20)

tweets= pd.read_csv("../Data/twpy2.csv") 

tweets

C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,date,tweet
0,2021-05-09 23:59:59,RT @faizaufi: luar negri: bobol data pribadi p...
1,2021-05-09 23:59:59,"RT @bukansaurusss: ""4 orang terkaya di indones..."
2,2021-05-09 23:59:59,RT @tribunnews: Harta Kekayaan Bupati Nganjuk ...
3,2021-05-09 23:59:58,RT @satriohendri: Sebetulnya target TKA asal C...
4,2021-05-09 23:59:57,@FahryBakrie @msaid_didu @detikcom Demen bener...
...,...,...
39995,2021-05-16 13:52:30,RT @DiajengLrst: Tingkat stress orang indonesi...
39996,2021-05-16 13:52:29,RT @billie9eulis: @MonicaChrista Soalnya di In...
39997,2021-05-16 13:52:29,RT @ronavioleta: Jokowi Ajak Erdogan dan Sejum...
39998,2021-05-16 13:52:27,RT @Adriandhy: Waktu Indonesia bagian hah libu...


In [10]:
class Preprocessing(object):
   
    def __init__(self):
        self.stopwords = stopwords.words('indonesian')
        self.ignored_word = ['rt','sih', 'loh', 'psx', 'ini', 'itu','eh', 'nya','si']
        self.append_stopwords(self.ignored_word)
            
    def lowercase(self, text):
        """ Lowercase text """
        return text.lower()
    
    def append_stopwords(self, text):
        """ Append Custom Stopword """
        for word in text:
            self.stopwords.append(word)
 
    def remove_newlines(self, text):
        """ Remove newlines characters from text """
        return text.replace("\n", "")
    
   
    def remove_tabs(self,text):
        """ Remove tab characters from text """
        return text.replace("\t", "")
    
   
    def remove_digits(self,text):
        """ Remove digits from text """
        return ''.join([word for word in text if not word.isdigit()])
    
    def remove_url(self,text):
        """ Remove urls from text """
        tokens = tweet_tokenizer.tokenize(text)
        output_array = []
        for word in tokens:
            cleaned_word = re.sub(r'^https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE)
            output_array.append(cleaned_word)
        return " ".join(output_array)
        
        
    def replace_special_chars(self,text):
        """ Remove special characters from text """
        return re.sub(r'\W+', ' ', text)

    
    def remove_multiple_whitespaces(self,text):
        """ Remove whitespaces from text """
        return re.sub(r'\s+', ' ', text, flags=re.I)

   
    def remove_trailing_whitespaces(self,text):
        """ Remove whitespaces from beggining and ending text """
        return text.strip()
     
    def lemmatize(self, text):
        """ Lemmatize words in text """          
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        text = stemmer.stem(text)
        return text

    def remove_stopwords(self, text):
        """ Remove stopwords from text 
        
        :param text: text
        :return: filtered stopwords from text
        """
        tokens = tweet_tokenizer.tokenize(text)
        filtered_tokens = [token for token in tokens if token.lower() not in self.stopwords]
        text = " ".join(filtered_tokens)
        return text
    
    def fix_slang(self, text, slang):
        """ Fix slang words from text 
        
        :param text: text
        :return: filtered slang from text
        """
        tokens = tweet_tokenizer.tokenize(text)

        filtered_tokens = []
        gotcha = 0

        for token in tokens:
            for index, row in slang.iterrows():
                if token == row["slang"]:
                    filtered_tokens.append(row["formal"])
                    gotcha = 1
        
            if gotcha == 0:
                filtered_tokens.append(token)
            else:
                gotcha = 0
            
        text = " ".join(filtered_tokens)

        return text
       
    def clean_text(self, text, slang):
        """ Generate a clean textual content"""
        text = self.remove_url(text)
        text = self.lowercase(text)
        text = self.remove_newlines(text)
        text = self.remove_tabs(text)
        text = self.remove_digits(text)
        text = self.replace_special_chars(text)
        text = self.remove_multiple_whitespaces(text)
        text = self.remove_trailing_whitespaces(text)        
        text = self.fix_slang(text, slang)
        text = self.lemmatize(text)
        text = self.remove_stopwords(text)        
        return text
    
    def clean_text_ws(self, text, slang):
        """ Generate a clean textual content"""
        text = self.remove_url(text)
        text = self.lowercase(text)
        text = self.remove_newlines(text)
        text = self.remove_tabs(text)
        text = self.remove_digits(text)
        text = self.replace_special_chars(text)
        text = self.remove_multiple_whitespaces(text)
        text = self.remove_trailing_whitespaces(text)        
        text = self.fix_slang(text, slang)     
        text = self.lemmatize(text)
        return text
    
    def process_text(self, data, slang):
        result_sentence=[]
        result_sentence_ws=[]
        result_tokens=[] 
        for sentence in data:
            clean_sentence = self.clean_text(sentence, slang)
            result_sentence.append(clean_sentence)
        return  result_sentence

In [11]:
processed_data = tweets
prep = Preprocessing()
data_clean = prep.process_text(processed_data["tweet"], slang)

processed_data['clean_sentences'] = data_clean

processed_data


,date,tweet,clean_sentences
0,2021-05-09 23:59:59,RT @faizaufi: luar negri: bobol data pribadi p...,faizaufi negri bobol data pribadi duduk hack w...
1,2021-05-09 23:59:59,"RT @bukansaurusss: ""4 orang terkaya di indones...",bukansaurusss orang kaya indonesia kaya tara j...
2,2021-05-09 23:59:59,RT @tribunnews: Harta Kekayaan Bupati Nganjuk ...,tribunnews harta kaya bupati nganjuk jaring ot...
3,2021-05-09 23:59:58,RT @satriohendri: Sebetulnya target TKA asal C...,satriohendri target tka china masuk masuk indo...
4,2021-05-09 23:59:57,@FahryBakrie @msaid_didu @detikcom Demen bener...,fahrybakrie msaid didu detikcom suka partai ko...
...,...,...,...
39995,2021-05-16 13:52:30,RT @DiajengLrst: Tingkat stress orang indonesi...,diajenglrst tingkat stress orang indonesia pan...
39996,2021-05-16 13:52:29,RT @billie9eulis: @MonicaChrista Soalnya di In...,billieeulis monicachrista indonesia waris buda...
39997,2021-05-16 13:52:29,RT @ronavioleta: Jokowi Ajak Erdogan dan Sejum...,ronavioleta jokowi ajak erdogan pimpin negara ...
39998,2021-05-16 13:52:27,RT @Adriandhy: Waktu Indonesia bagian hah libu...,adriandhy indonesia hah libur


In [12]:
processed_data.to_csv('../Data/twpy_pre2.csv',  index = False)

In [ ]:
processed_data.to_csv('drive/My Drive/Thesis/Data/small_pre_tweets_4.csv', mode = 'a', header = False,  index = False)